In [ ]:
!pip install -U google-genai
!pip install Pillow

In [ ]:
import os
from google import genai
from PIL import Image
from io import BytesIO
from IPython.display import display
from dotenv import load_dotenv
load_dotenv()

# --- IMPORTANT ---
# Paste your API key here. For better security, we recommend using environment variables.
# For example: API_KEY=os.environ.get("GEMINI_API_KEY")
API_KEY = os.getenv("API_KEY")
# -----------------

# Configure the client with your API key
client = genai.Client(api_key=API_KEY)

NANO_BANANA = "gemini-2.5-flash-image-preview"

In [ ]:
from IPython.display import display, Markdown, Image
import pathlib

# Loop over all parts and display them either as text or images
def display_response(response):
  for part in response.parts:
    if part.text:
      display(Markdown(part.text))
    elif image:= part.as_image():
      display(image)
      # image.show() if not in a notebook

# Save the image
# If there are multiple ones, only the last one will be saved
def save_image(response, path):
  for part in response.parts:
    if image:= part.as_image():
      image.save(path)

## Image Generation

In [ ]:
# The text prompt for image generation
prompt = "Create a photorealistic image of an orange cat with green eyes, sitting on a couch."

print("Generating image...")

# Call the API to generate the image
response = client.models.generate_content(
    model=NANO_BANANA,
    contents=prompt,
)

image_parts = [
    part.inline_data.data
    for part in response.candidates[0].content.parts
    if part.inline_data
]
 
if image_parts:
    image = Image.open(BytesIO(image_parts[0]))
    image.save('cat.png')
    display(image)

## Image Editing

In [ ]:
text_prompt = "Create a side view picture of that cat, in a tropical forest, eating a nano-banana, under the stars" # @param {type:"string"}

response = client.models.generate_content(
    model=NANO_BANANA,
    contents=[
        text_prompt,
        Image.open('cat.png')
    ]
)


## Generate multiple images at once

In [ ]:
prompt = "Create a beautifully entertaining 8 part story with 8 images with two blue characters and their adventures in the 1960s music scene. The story is thrilling throughout with emotional highs and lows and ending on a great twist and high note. Do not include any words or text on the images but tell the story purely through the imagery itself. " # @param ["Show me how to bake macarons with images","Create a beautifully entertaining 8 part story with 8 images with two blue characters and their adventures in the 1960s music scene. The story is thrilling throughout with emotional highs and lows and ending on a great twist and high note. Do not include any words or text on the images but tell the story purely through the imagery itself. "] {"allow-input":true}

response = client.models.generate_content(
    model=NANO_BANANA,
    contents=prompt,
)

display_response(response)


## Structured Output

In [ ]:
from pydantic import BaseModel

class Recipe(BaseModel):
    recipe_name: str
    ingredients: list[str]

client = genai.Client()
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="List a few popular cookie recipes, and include the amounts of ingredients.",
    config={
        "response_mime_type": "application/json",
        "response_schema": list[Recipe],
    },
)
# Use the response as a JSON string.
print(response.text)

# Use instantiated objects.
my_recipes: list[Recipe] = response.parsed